In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

def entrenar_regresion_lineal(df_modelo, features, test_size=0.2, shuffle=False):
    """
    Aplica regresión lineal con escalado de features.
    Devuelve predicciones, métricas y el modelo entrenado.
    """
    # Validación de columnas necesarias
    columnas_necesarias = features + ['Dias Cama Disponibles']
    faltantes = [col for col in columnas_necesarias if col not in df_modelo.columns]
    if faltantes:
        raise ValueError(f"Columnas faltantes en df_modelo: {faltantes}")

    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=shuffle, test_size=test_size)

    # Pipeline con escalado + modelo
    modelo = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])

    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    print(f"✅ MAE (Reg. Lineal): {mae:.2f}")
    print(f"✅ RMSE (Reg. Lineal): {rmse:.2f}")

    return resultados, modelo

# Ejecución del modelo
predicciones_lr, modelo_lr = entrenar_regresion_lineal(df_modelo, features)
display(predicciones_lr)


In [ ]:
def preparar_datos_para_modelo(df, hospitales_validos, anio=2024):
    """
    Transforma el dataset original de formato mensual a formato largo diario.
    Incluye solo hospitales que están en la lista de hospitales válidos.

    Devuelve un DataFrame con columnas:
    ['Fecha', 'Nombre Establecimiento', 'Nombre Nivel Cuidado', 'Glosa', 'Valor']
    """
    # 1. Nos quedamos solo con los hospitales válidos especificados
    df = df[df['Nombre Establecimiento'].isin(hospitales_validos)].copy()

    if df.empty:
        raise ValueError("No hay hospitales válidos en el DataFrame.")

    # 2. Diccionario para traducir meses a números
    meses_mapeo = {
        'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Ago': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
    }

    columnas_meses = list(meses_mapeo.keys())

    # 3. Transformar de ancho a largo
    df_largo = df.melt(
        id_vars=['Nombre Establecimiento', 'Nombre Nivel Cuidado', 'Glosa'],
        value_vars=columnas_meses,
        var_name='Mes',
        value_name='Valor'
    )

    # 4. Crear fecha usando el día 15 por simplicidad
    df_largo['Mes_Num'] = df_largo['Mes'].map(meses_mapeo)
    df_largo['Fecha'] = pd.to_datetime({'year': anio, 'month': df_largo['Mes_Num'], 'day': 15})

    # 5. Filtrar y asegurar formato numérico
    df_largo = df_largo[['Fecha', 'Nombre Establecimiento', 'Nombre Nivel Cuidado', 'Glosa', 'Valor']]
    df_largo['Valor'] = pd.to_numeric(df_largo['Valor'], errors='coerce')

    return df_largo

# Ejecución de la preparación de datos
df_modelo_base = preparar_datos_para_modelo(df_2024_csv, hospitales_lista)
df_modelo_base.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

def entrenar_modelo_con_glosas_extras(df, hospital, area):
    """
    Entrena un modelo robusto de predicción de 'Dias Cama Disponibles' para un hospital y área específicos.
    Utiliza como variables predictoras:
    - Información directamente extraída desde la glosa: 'Promedio Cama Disponibles', 'Numero de Egresos'
    - Variables temporales: mes y trimestre
    - Derivadas: media móvil, variación intermensual, porcentaje de ocupación, etc.
    """

    # 1. Filtrar por hospital y área
    df_filtro = df[
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    ].copy()

    if df_filtro.empty:
        return f"⚠️ No hay datos para {hospital} - {area}"

    # 2. Pivotear glosas a columnas
    df_pivot = df_filtro.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    # 3. Validar columnas mínimas
    columnas_necesarias = [
        'Dias Cama Disponibles', 'Dias Cama Ocupados',
        'Promedio Cama Disponibles', 'Numero de Egresos'
    ]
    faltantes = [col for col in columnas_necesarias if col not in df_pivot.columns]
    if faltantes:
        return f"❌ Faltan columnas necesarias en {hospital} - {area}: {faltantes}"

    # 4. Orden cronológico
    df_pivot = df_pivot.sort_values('Fecha')

    # 5. Variables temporales
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # 6. Variables derivadas
    df_pivot['promedio_mes_anterior'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['porcentaje_ocupacion'] = (
        df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    ).replace([np.inf, -np.inf], np.nan)
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()

    # 7. Limpieza de datos (eliminar NaNs generados por shift y rolling)
    df_modelo = df_pivot.dropna().copy()
    if df_modelo.empty:
        return f"⚠️ Sin datos suficientes tras ingeniería de variables en {hospital} - {area}"

    # 8. Definir X e y
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'promedio_mes_anterior',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil'
    ]
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    # 9. Split temporal
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    # 10. Modelo
    modelo = RandomForestRegressor(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)

    # 11. Predicción y evaluación
    y_pred = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # 12. Resultados
    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    return {
        'hospital': hospital,
        'area': area,
        'mae': mae,
        'rmse': rmse,
        'predicciones': resultados,
        'modelo': modelo
    }


In [ ]:
# ----------- EJECUCIÓN -------------

resultado_glosas = entrenar_modelo_con_glosas_extras(
    df_modelo_base,
    "Hospital Dr. Ernesto Torres Galdames (Iquique)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

# Mostrar resultados o advertencia
if isinstance(resultado_glosas, dict):
    display(resultado_glosas['predicciones'])
    print(f"✅ MAE: {resultado_glosas['mae']:.2f} | RMSE: {resultado_glosas['rmse']:.2f}")
else:
    print(resultado_glosas)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

def entrenar_modelo_gradient_boosting(df, hospital, area):
    """
    Entrena un modelo Gradient Boosting para predecir 'Dias Cama Disponibles'.
    Usa todas las variables disponibles desde glosa + derivadas + temporales.
    """

    # 1. Filtrar datos
    df_filtro = df[
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    ].copy()

    if df_filtro.empty:
        return f"⚠️ No hay datos para {hospital} - {area}"

    # 2. Pivotear glosas
    df_pivot = df_filtro.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    # 3. Validar columnas clave
    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    faltantes = [col for col in columnas_necesarias if col not in df_pivot.columns]
    if faltantes:
        return f"❌ Faltan columnas necesarias en {hospital} - {area}: {faltantes}"

    # 4. Orden cronológico
    df_pivot = df_pivot.sort_values('Fecha')

    # 5. Variables temporales
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # 6. Variables derivadas
    df_pivot['promedio_mes_anterior'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['porcentaje_ocupacion'] = (
        df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    ).replace([np.inf, -np.inf], np.nan)
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()

    # 7. Eliminar filas con NaNs
    df_modelo = df_pivot.dropna().copy()
    if df_modelo.empty:
        return f"⚠️ Sin datos suficientes tras ingeniería de variables en {hospital} - {area}"

    # 8. Definir variables predictoras y objetivo
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'promedio_mes_anterior',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil'
    ]
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    # 9. Split temporal
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    # 10. Entrenar modelo
    modelo = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    modelo.fit(X_train, y_train)

    # 11. Predicciones y métricas
    y_pred = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # 12. Construcción de resultados
    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    return {
        'modelo': 'GradientBoosting',
        'hospital': hospital,
        'area': area,
        'mae': mae,
        'rmse': rmse,
        'predicciones': resultados,
        'modelo_entrenado': modelo
    }


In [ ]:
# ----------- EJECUCIÓN -------------

resultado_gb = entrenar_modelo_gradient_boosting(
    df_modelo_base,
    "Hospital Dr. Ernesto Torres Galdames (Iquique)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

# Mostrar resultados o mensaje
if isinstance(resultado_gb, dict):
    display(resultado_gb['predicciones'])
    print(f"🌟 MAE: {resultado_gb['mae']:.2f} | RMSE: {resultado_gb['rmse']:.2f}")
else:
    print(resultado_gb)

In [ ]:
def entrenar_gradient_boosting_mejorado(df, hospital, area):
    """
    Entrena un modelo Gradient Boosting con más contexto temporal:
    - Lags de camas disponibles
    - Medias móviles de ocupados, promedio camas y egresos
    """

    # 1. Filtrar por hospital y área
    df_filtro = df[
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    ].copy()

    if df_filtro.empty:
        return f"⚠️ No hay datos para {hospital} - {area}"

    # 2. Pivotear glosas
    df_pivot = df_filtro.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    # 3. Validar columnas clave
    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    faltantes = [col for col in columnas_necesarias if col not in df_pivot.columns]
    if faltantes:
        return f"❌ Faltan columnas necesarias en {hospital} - {area}: {faltantes}"

    # 4. Orden cronológico
    df_pivot = df_pivot.sort_values('Fecha')

    # 5. Variables temporales
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # 6. Nuevas features: lags y medias móviles
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = (
        df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    ).replace([np.inf, -np.inf], np.nan)
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    # 7. Eliminar NaNs
    df_modelo = df_pivot.dropna().copy()
    if df_modelo.empty:
        return f"⚠️ Sin datos suficientes tras ingeniería de variables en {hospital} - {area}"

    # 8. Features y target
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil'
    ]
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    # 9. División temporal
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    # 10. Entrenar modelo
    modelo = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    modelo.fit(X_train, y_train)

    # 11. Predicción y métricas
    y_pred = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # 12. Resultados
    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    return {
        'modelo': 'GradientBoosting (con lags y medias móviles)',
        'hospital': hospital,
        'area': area,
        'mae': mae,
        'rmse': rmse,
        'predicciones': resultados,
        'modelo_entrenado': modelo
    }


In [ ]:
# ----------- EJECUCIÓN -------------

resultado_gb_mejorado = entrenar_gradient_boosting_mejorado(
    df_modelo_base,
    "Hospital Dr. Ernesto Torres Galdames (Iquique)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

if isinstance(resultado_gb_mejorado, dict):
    display(resultado_gb_mejorado['predicciones'])
    print(f"🌟 MAE: {resultado_gb_mejorado['mae']:.2f} | RMSE: {resultado_gb_mejorado['rmse']:.2f}")
else:
    print(resultado_gb_mejorado)

In [ ]:
from xgboost import XGBRegressor

def entrenar_modelo_xgboost_optimo(df, hospital, area):
    """
    Entrena un modelo XGBoost más liviano y optimizado en tiempo de ejecución.
    """

    # 1. Filtrar por hospital y área
    df_filtro = df[
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    ].copy()

    if df_filtro.empty:
        return f"⚠️ No hay datos para {hospital} - {area}"

    # 2. Pivotear glosas
    df_pivot = df_filtro.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    # 3. Validar columnas necesarias
    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    faltantes = [col for col in columnas_necesarias if col not in df_pivot.columns]
    if faltantes:
        return f"❌ Faltan columnas necesarias en {hospital} - {area}: {faltantes}"

    # 4. Ordenar por fecha
    df_pivot = df_pivot.sort_values('Fecha')

    # 5. Variables temporales
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # 6. Features históricas
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = (
        df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    ).replace([np.inf, -np.inf], np.nan)
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    # 7. Limpiar datos nulos
    df_modelo = df_pivot.dropna().copy()
    if df_modelo.empty:
        return f"⚠️ Sin datos suficientes tras ingeniería de variables en {hospital} - {area}"

    # 8. Features y target
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil'
    ]
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    # 9. Train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    # 10. Entrenar XGBoost optimizado
    modelo = XGBRegressor(
        n_estimators=50,
        max_depth=3,
        learning_rate=0.1,
        random_state=42,
        n_jobs=1,
        verbosity=0
    )
    modelo.fit(X_train, y_train)

    # 11. Predicción y evaluación
    y_pred = modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # 12. Resultados
    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    return {
        'modelo': 'XGBoost (optimizado)',
        'hospital': hospital,
        'area': area,
        'mae': mae,
        'rmse': rmse,
        'predicciones': resultados,
        'modelo_entrenado': modelo
    }


In [ ]:
# ----------- EJECUCIÓN -------------

resultado_xgb_optimo = entrenar_modelo_xgboost_optimo(
    df_modelo_base,
    "Hospital Dr. Ernesto Torres Galdames (Iquique)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

if isinstance(resultado_xgb_optimo, dict):
    display(resultado_xgb_optimo['predicciones'])
    print(f"🌟 MAE: {resultado_xgb_optimo['mae']:.2f} | RMSE: {resultado_xgb_optimo['rmse']:.2f}")
else:
    print(resultado_xgb_optimo)


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

def afinar_lightgbm(df_modelo, features):
    """
    Ajusta un modelo LightGBM usando validación cruzada temporal (TimeSeriesSplit).
    """

    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    tscv = TimeSeriesSplit(n_splits=3)

    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    model = LGBMRegressor(random_state=42)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=tscv,
        verbose=1
    )

    grid_search.fit(X, y)

    print("📌 Mejores parámetros encontrados (LightGBM):", grid_search.best_params_)

    return grid_search.best_estimator_

def evaluar_modelo(modelo, df_modelo, features):
    """
    Evalúa un modelo entrenado usando MAE y RMSE sobre los últimos datos del conjunto.
    """

    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred
    })

    print(f"✅ MAE: {mae:.2f}")
    print(f"✅ RMSE: {rmse:.2f}")

    return resultados

# Ya debes tener definido df_modelo y features como hicimos antes

# Ejecutar LightGBM si df_modelo existe
if df_modelo is not None:
    mejor_lgbm = afinar_lightgbm(df_modelo, features)
    predicciones_lgbm = evaluar_modelo(mejor_lgbm, df_modelo, features)
    display(predicciones_lgbm)

In [ ]:
# Cargar el archivo de clima
df_clima = pd.read_csv('todos_los_seremi_2014.csv')

# Asegurarnos de que la columna 'Mes' esté en formato datetime para facilitar el merge
df_clima['Mes'] = pd.to_datetime(df_clima['Mes'], format='%Y-%m')

# Verificar que se cargaron correctamente
df_clima.head()


In [ ]:
# Suponiendo que 'df_modelo' tiene la columna 'Fecha'
df_modelo['Mes'] = df_modelo['Fecha'].dt.to_period('M')

# Unir los datos de clima con los datos hospitalarios
df_modelo_completo = pd.merge(df_modelo, df_clima, on='Mes', how='left')


In [ ]:
# Agregar las columnas de clima a las features
features.extend([
    'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'winddirection_10m_dominant'
])

# Ahora puedes usar 'features' con las variables de clima al entrenar el modelo


In [ ]:
if df_modelo_completo is not None:
    mejor_xgb = afinar_xgboost(df_modelo_completo, features)
    predicciones = evaluar_modelo(mejor_xgb, df_modelo_completo, features)
    display(predicciones)


In [ ]:
def preparar_dataset(df, hospital, area, df_clima):
    df_filtro = df[
        (df['Nombre Establecimiento'] == hospital) &
        (df['Nombre Nivel Cuidado'] == area)
    ].copy()

    df_pivot = df_filtro.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    columnas_necesarias = [
        'Dias Cama Disponibles', 'Dias Cama Ocupados',
        'Promedio Cama Disponibles', 'Numero de Egresos'
    ]
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        print(f"Datos insuficientes para {hospital} - {area}")
        return None

    # Agregamos mes y trimestre
    df_pivot = df_pivot.sort_values('Fecha')
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    # Clima: extraer año-mes y unir con df_clima
    df_pivot['Mes_str'] = df_pivot['Fecha'].dt.to_period('M').astype(str)
    region = df_filtro['SEREMI'].iloc[0]  # asumimos que todo viene de la misma región
    df_clima_region = df_clima[df_clima['SEREMI'] == region].copy()
    df_clima_region.rename(columns={'Mes': 'Mes_str'}, inplace=True)

    df_pivot = df_pivot.merge(df_clima_region, on='Mes_str', how='left')

    # Variables adicionales
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    # Columnas de clima
    columnas_clima = [
        'temperature_2m_max',
        'temperature_2m_min',
        'precipitation_sum',
        'winddirection_10m_dominant'
    ]
    if not all(col in df_pivot.columns for col in columnas_clima):
        print(f"❌ Datos climáticos incompletos para la región: {region}")
        return None

    return df_pivot.dropna().copy()


In [ ]:
df_clima = pd.read_csv("todos_los_seremi_2014.csv")
df_modelo = preparar_dataset(df_modelo_base, hospital, area, df_clima)